In [1]:
import torch
import torch.nn as nn
from torch import optim

#### Load data

In [2]:
device = torch . device ('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
noisy_imgs_1, noisy_imgs_2 = torch.load('../data/train_data.pkl')
noisy_imgs, clean_imgs = torch.load('../data/val_data.pkl')

In [16]:
img_C = noisy_imgs_1.shape[0]
img_W = noisy_imgs_1.shape[2]
img_H = noisy_imgs_1.shape[3]

In [22]:
### For mini - project 1
class Model ():
    def __init__(self) -> None :
        # instantaiate model + optimizer + loss function + any other stuff you need

        # model: UNet - first idea: use alternate convd2 with relu activation and leakyrelu at the end
        self.model = nn.Sequential(
            nn.Conv2d(img_C, 64, 3, stride=1, padding=1),
            nn.ReLU(inplace=True), 
            nn.Conv2d(64, 48, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(48, 48, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(48, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, img_C, 3, stride=1, padding=1),
            nn.LeakyReLU(0.1)
        )
        # can also use upsampling2d layer or7and maxpooling2d for decoding
        
        # optimizer: Adam; try different learning rates
        self.optim = optim.Adam(self.model.parameters(), lr = 1e-3)

        # loss function: MSE or HDRLoss for MonteCarlo images ? 
        self.loss = nn.MSELoss()

    def load_pretrained_model(self) -> None :
        ## This loads the parameters saved in bestmodel .pth into the model
        pass

    def train(self, train_input, train_target, num_epochs) -> None :
        #: train˙input : tensor of size (N, C, H, W) containing a noisy version of the images
        #: train˙target : tensor of size (N, C, H, W) containing another noisy version of the same images , which only differs from the input by their noise .
        mini_batch_size = 100

        for e in range(num_epochs):
            for b in range(0, train_input.size(0), mini_batch_size):
                output = self.model(train_input.narrow(0, b, mini_batch_size))
                loss = self.loss(output, train_target.narrow(0, b, mini_batch_size))
                self.model.zero_grad()
                loss.backward()
                self.optimizer.step()

    def predict(self, test_input ) -> torch.Tensor:
        #:test_input : tensor of size (N1 , C, H, W) that has to be denoised by the trained or the loaded network .
        #: returns a tensor of the size (N1 , C, H, W)
        pass

In [21]:
model = Model()
model.train(noisy_imgs_1, noisy_imgs_2)

TypeError: train() missing 1 required positional argument: 'num_epochs'